# Minimal CASteer Tutorial
Demonstrates activation steering with Stable Diffusion XL.

## Setup
Load the pipeline and helper functions.

In [ ]:
import os
import re
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os


In [ ]:
!git clone https://github.com/sidhantls/minimal-casteer
%cd minimal-casteer
! git pull origin main

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
import steering
import prompt_catalog

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    generator=torch.Generator().manual_seed(0),
)
pipe.to("cuda")
pipe.set_progress_bar_config(disable=True)


## Hyperparameters
Basic settings for steering.

In [ ]:
STEER_TYPE = "default"
INF_STEPS = 20
GUIDE_SCALE = 5.0
PROMPT_THEME = "anime" # can be metal or anime


## Build and apply steering vectors

To guide a model’s generation toward or away from a target attribute, we will construct **steering vectors**. The process involves four steps:

1. **Create prompt pairs**  
   We start by defining two sets of prompts:  
   - **Positive prompts** that contain the target attribute (e.g., “box turtle, anime style").  
   - **Baseline prompts** that are matched but neutral (e.g., "box turtle").  
   These pairs allow us to isolate the effect of the attribute by controlling for context.

2. **Cache conditional activations**  
   For each prompt, we record the model’s hidden activations at a chosen layer. This caching step is crucial because we want to compare how the internal representations differ between positive and baseline prompts. Efficient caching avoids recomputation during training or experimentation.

3. **Calculate steering vectors**  
   The steering vector is defined as the difference between the average hidden activations of the positive and baseline prompts:  
$$
v = \frac{1}{N} \sum_{i=1}^N h(x_i^{+}) - \frac{1}{M} \sum_{j=1}^M h(x_j^{-})
$$


   where \(h(x)\) is the activation of prompt \(x\) at the chosen layer, \(x_i^{+}\) are positive prompts, and \(x_j^{-}\) are baseline prompts.  
   This vector \(v\) represents the “direction” in activation space corresponding to the target attribute ([Meng et al., 2025](https://arxiv.org/abs/2503.09630)).

4. **Register steering vectors**  
   Finally, we apply the steering vector during generation. At each forward pass, we adjust the hidden activations by adding or subtracting a scaled version of \(v\):  

$$
   h'(x) = h(x) + \alpha v
$$


   where $\alpha$ controls the strength of steering. Positive $\alpha$ amplifies the attribute, while negative $\alpha$ suppresses it.


### Create prompt pairs
Each pair contains a steered prompt and a neutral prompt. Feel free to swap in your own themes.


In [ ]:
if PROMPT_THEME == "anime":
    prompts = prompt_catalog.ANIME_PROMPT[:20]
elif PROMPT_THEME == "metal":
    prompts = prompt_catalog.METALLIC_SCULPTURE_SET[:20]
else:
    raise NotImplementedError(f"Prompt theme {PROMPT_THEME} not implemented")

print('Example of the prompts:\n')
for p in prompts[:5]:
  print(p)


### Attach steer hooks
`add_steer_hooks` registers forward hooks on attention layers to cache activations.


In [ ]:
steer_hooks = steering.add_steer_hooks(
    pipe,
    steer_type=STEER_TYPE,
    save_every=1,
)


### Build steering vector
Run the prompt pairs through the model to compute the conditional activation differences.


In [ ]:
final_vecs = steering.build_final_steering_vectors(
    pipe,
    steer_hooks,
    prompts,
    num_inference_steps=INF_STEPS,
    guidance_scale=GUIDE_SCALE,
)
print(final_vecs[0].shape)


### Register steering vector
Attach the computed vectors so they influence future generations.


In [ ]:
steering.add_final_steer_vectors(steer_hooks, final_vecs)


## Generate images
Apply the steering vectors to new prompts and compare results.

In [ ]:
TEST_PROMPTS = [
    "Studio-lit Batman logo on a plain background, 4K detail.",
    "Cinematic close-up of Batman's face, dramatic shadows across the cowl, ultra-detailed, 4K.",
    "Close-up of a labrador sitting.",
    "Portrait of an Apple.",
    "Close-up of a amicable robot face, made of plastic, with a neural expression.",
    "Portrait of a carved pumpkin jack-o’-lantern with a neural expression, dramatic lighting.",
    "Cinematic portrait of an astronaut with reflective visor, stars and Earth visible, ultra-detailed.",
    "Minimalist black-and-white sketch of a cat stretching.",
]

STEER_SCALE_LIST = [0.0, 2.0, 5.0, 10.0] # various steering strengths. 0.0 is baseline image
steering.run_grid_experiment(
    pipe,
    steer_hooks,
    TEST_PROMPTS,
    num_inference_steps=INF_STEPS,
    steer_type=STEER_TYPE,
    gscale_list=[GUIDE_SCALE],
    steer_scale_list=STEER_SCALE_LIST,
    out_root=f"{PROMPT_THEME}_experiments",
)


## Plot Images

### Utils

In [ ]:
import os
import re
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def extract_steer_value(folder_name: str) -> float:
    """Extracts the steer value from a folder name like steps=20_guide=5.0_steer=0.5_type=default."""
    match = re.search(r"steer=([0-9]*\.?[0-9]+)", folder_name)
    return float(match.group(1)) if match else float("inf")

def plot_all_param_images(base_folder: str, save_dir: str = "plots_output"):
    """
    For each unique filename across param folders, plot horizontally across all
    steer values and save the plot.

    Args:
        base_folder (str): Path to the experiments folder (e.g. 'metal_experiments').
        save_dir (str): Folder where plots will be saved.

    Returns:
        List[str]: Filenames of saved plots.
    """
    os.makedirs(save_dir, exist_ok=True)

    # list all param subfolders
    param_folders = [os.path.join(base_folder, f) for f in os.listdir(base_folder)
                     if os.path.isdir(os.path.join(base_folder, f))]

    if not param_folders:
        raise ValueError("No param folders found in base folder.")

    # sort by steer value
    param_folders.sort(key=lambda x: extract_steer_value(os.path.basename(x)))

    # collect all unique filenames from the first folder
    sample_files = [f for f in os.listdir(param_folders[0]) if f.lower().endswith((".jpg", ".png"))]
    if not sample_files:
        raise ValueError("No images found in param folders.")

    saved_plots = []

    # loop through all unique filenames
    for chosen_file in sample_files:
        images = []
        labels = []
        for pf in param_folders:
            img_path = os.path.join(pf, chosen_file)
            if os.path.exists(img_path):
                images.append(mpimg.imread(img_path))
                labels.append(os.path.basename(pf))

        if not images:
            continue

        # plot horizontally
        n = len(images)
        plt.figure(figsize=(4*n, 4))
        for i, (img, label) in enumerate(zip(images, labels)):
            plt.subplot(1, n, i+1)
            plt.imshow(img)
            plt.axis("off")
            steer_val = extract_steer_value(label)
            plt.title(f"$\\alpha$={steer_val}", fontsize=10)  # alpha symbol

        # add main title = filename
        # plt.suptitle(chosen_file, fontsize=12, y=1.02)

        plt.tight_layout(rect=[0, 0, 1, 0.95])  # prevent clipping

        # save figure
        save_filename = f"{os.path.splitext(chosen_file)[0]}_comparison.png"
        save_path = os.path.join(save_dir, save_filename)
        plt.savefig(save_path, bbox_inches="tight")
        plt.close()

        saved_plots.append(save_filename)

    print(f"All plots saved in: {os.path.abspath(save_dir)}")
    return saved_plots

def list_saved_plots(save_dir: str = "plots_output"):
    """List all saved plots in the output directory."""
    if not os.path.exists(save_dir):
        raise FileNotFoundError(f"{save_dir} not found.")
    plots = [f for f in os.listdir(save_dir) if f.endswith("_comparison.png")]
    print("Available plots:", plots)
    return plots

def load_saved_plot(filename: str, save_dir: str = "plots_output"):
    """
    Load and display a saved plot by filename.

    Args:
        filename (str): Filename of saved plot (e.g. '00001_comparison.png').
        save_dir (str): Directory where plots were saved.
    """
    file_path = os.path.join(save_dir, filename)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found.")

    img = mpimg.imread(file_path)
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.show()


### Plot figures

In [ ]:
# Example usage:
folder = f"{PROMPT_THEME}_experiments"
save_dir = os.path.join(folder, "new_outputs")

# Generate and save all comparison plots
saved = plot_all_param_images(folder, save_dir=save_dir)

### Load plot

In [ ]:
# List what was saved
available = list_saved_plots(save_dir)
path = random.sample(available, 1)[0]

print(path)

# Load one saved plot
if available:
    load_saved_plot(path, save_dir=save_dir)